In [0]:
# modeling imports
import tensorflow as tf
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Conv1D, MaxPooling1D, Flatten
from keras.layers import Input, Dense
from keras.layers.merge import Add, Dot, Concatenate

# preprocessing imports
import pandas as pd
import numpy as np
from time import time

Using TensorFlow backend.


In [0]:
from keras.models import Model
from keras.layers import Input, Dense
class StackedNN():
  def __init__(self,train_user,train_business,alpha,output_dim=5):
        
        # the shape of one training example is

        user_inp, user_model = self.createModel(train_user)
        business_inp, business_model = self.createModel(train_business)
        stacked_layer = Concatenate()([user_model,business_model])
        output_layer =  Dense(32, activation='relu')(stacked_layer)
        output_layer = Dense(output_dim,activation='softmax')(output_layer)
        self.model = Model(inputs=[user_inp, business_inp],outputs=output_layer)
        self.model.compile(optimizer=keras.optimizers.Adam(alpha), loss='categorical_crossentropy', metrics=['accuracy'])
        
        
  def createModel(self,training_data):
        input_shape = training_data.shape[1]
        inp_layer = Input(shape=(input_shape,))
        dense_layer = Dense(units=64, activation='relu')(inp_layer)
        dense_layer = Dense(units=32, activation='relu')(dense_layer)

        #model.add(Dense(units=5, activation='softmax'))
        return inp_layer, dense_layer
   
  def train(self,train_user,train_business,output_rating,num_iter):
        train_inputs = [train_user, train_business] 
        self.model.fit(train_inputs, output_rating, epochs=num_iter)
  
 
   
  
      


In [0]:
import pandas as pd
from keras.utils import to_categorical
import numpy as np
from keras.layers import Concatenate
import keras

In [0]:
train_user = pd.read_csv("LDA Modelling/results/train/user_FeatureVec.csv").drop(["user_id","review_id","business_id","1overN","2overN","percentile"],axis=1).fillna(0)
train_business = pd.read_csv("LDA Modelling/results/train/business_FeatureVec.csv").drop(["business_id","review_id","user_id","1overN","2overN","percentile"],axis=1).fillna(0)
#train_business = train_business.loc[:,["topic_0","topic_1","topic_2","topic_3","topic_4","topic_5","topic_6","topic_7","topic_8","topic_9","topic_10","topic_11","topic_12","topic_13","topic_14","topic_15","topic_16","topic_17","topic_18","topic_19","business_stars","is_open","attributes.Ambience.casual","attributes.BikeParking","attributes.BusinessAcceptsCreditCards","attributes.BusinessParking.street","attributes.HasTV","attributes.RestaurantsDelivery","attributes.GoodForMeal.breakfast","attributes.GoodForMeal.brunch","attributes.GoodForMeal.dessert","attributes.GoodForMeal.dinner","attributes.GoodForMeal.latenight","attributes.GoodForMeal.lunch","attributes.Alcohol","attributes.NoiseLevel","attributes.RestaurantsPriceRange2"]]
stacked= train_business[['attributes.Ambience.casual','attributes.BikeParking','attributes.BusinessAcceptsCreditCards','attributes.BusinessParking.street','attributes.HasTV','attributes.RestaurantsDelivery','attributes.GoodForMeal.breakfast','attributes.GoodForMeal.brunch','attributes.GoodForMeal.dessert','attributes.GoodForMeal.dinner','attributes.GoodForMeal.latenight','attributes.GoodForMeal.lunch','attributes.Alcohol','attributes.NoiseLevel']].stack()
train_business[['attributes.Ambience.casual','attributes.BikeParking','attributes.BusinessAcceptsCreditCards','attributes.BusinessParking.street','attributes.HasTV','attributes.RestaurantsDelivery','attributes.GoodForMeal.breakfast','attributes.GoodForMeal.brunch','attributes.GoodForMeal.dessert','attributes.GoodForMeal.dinner','attributes.GoodForMeal.latenight','attributes.GoodForMeal.lunch','attributes.Alcohol','attributes.NoiseLevel']]=pd.Series(stacked.factorize()[0], index=stacked.index).unstack()
#reviews = pd.read_csv("user_business_reviewVec.csv")['review_id']

rating_csv = pd.read_csv("Dataset/PA/Restaurants/train/PA_train_yelp_academic_dataset_review.csv")
rating_csv_stars = rating_csv['stars']

encoded_rating = to_categorical(np.array(list(rating_csv_stars)))
encoded_rating_ = encoded_rating[:,1:]
stackedModel = StackedNN(train_user,train_business,0.00000001,5)


In [0]:
stackedModel.train(train_user,train_business,encoded_rating_,500)

Epoch 1/500
84887/84887 [==============================] - 11s 133us/step - loss: 2.2314 - acc: 0.1915
Epoch 2/500
84887/84887 [==============================] - 10s 116us/step - loss: 2.2302 - acc: 0.1916
Epoch 3/500
84887/84887 [==============================] - 10s 113us/step - loss: 2.2290 - acc: 0.1917
Epoch 4/500
84887/84887 [==============================] - 9s 108us/step - loss: 2.2278 - acc: 0.1919
Epoch 5/500
84887/84887 [==============================] - 11s 124us/step - loss: 2.2266 - acc: 0.1919
Epoch 6/500
84887/84887 [==============================] - 10s 116us/step - loss: 2.2254 - acc: 0.1920
Epoch 7/500
84887/84887 [==============================] - 10s 112us/step - loss: 2.2242 - acc: 0.1922
Epoch 8/500
84887/84887 [==============================] - 10s 112us/step - loss: 2.2231 - acc: 0.1922
Epoch 9/500
84887/84887 [==============================] - 9s 109us/step - loss: 2.2219 - acc: 0.1924
Epoch 10/500
84887/84887 [==============================] - 9s 110us/step -

In [0]:
stackedModel.model.save('model_1_12_stars.h5')

In [0]:
from google.colab import files
files.download('model_1_12_stars.h5')

In [0]:
stackedModel.model.save_weights('model__weights_1_12_stars.h5')
files.download('model__weights_1_12_stars.h5')

In [0]:
valid_user = pd.read_csv("user_FeatureVec_valid.csv").drop(["user_id","review_id","business_id","1overN","2overN","percentile"],axis=1).fillna(0)
valid_business = pd.read_csv("business_FeatureVec_valid.csv").drop(["business_id","review_id","user_id","1overN","2overN","percentile"],axis=1).fillna(0)
stacked= valid_business[['attributes.Ambience.casual','attributes.BikeParking','attributes.BusinessAcceptsCreditCards','attributes.BusinessParking.street','attributes.HasTV','attributes.RestaurantsDelivery','attributes.GoodForMeal.breakfast','attributes.GoodForMeal.brunch','attributes.GoodForMeal.dessert','attributes.GoodForMeal.dinner','attributes.GoodForMeal.latenight','attributes.GoodForMeal.lunch','attributes.Alcohol','attributes.NoiseLevel']].stack()
valid_business[['attributes.Ambience.casual','attributes.BikeParking','attributes.BusinessAcceptsCreditCards','attributes.BusinessParking.street','attributes.HasTV','attributes.RestaurantsDelivery','attributes.GoodForMeal.breakfast','attributes.GoodForMeal.brunch','attributes.GoodForMeal.dessert','attributes.GoodForMeal.dinner','attributes.GoodForMeal.latenight','attributes.GoodForMeal.lunch','attributes.Alcohol','attributes.NoiseLevel']]=pd.Series(stacked.factorize()[0], index=stacked.index).unstack()
#reviews = pd.read_csv("user_business_reviewVec.csv")['review_id']

valid_rating_csv = pd.read_csv("Dataset/PA/Restaurants/valid/PA_valid_yelp_academic_dataset_review.csv")
valid_rating_csv_stars = valid_rating_csv['stars']
encoded_rating = to_categorical(np.array(list(valid_rating_csv_stars)))
encoded_rating_valid = encoded_rating[:,1:]


In [0]:
valid_user.shape

(27255, 32)

In [0]:
test_user = pd.read_csv("LDA Modelling/results/test/user_FeatureVec_test.csv").drop(["user_id","review_id","business_id","1overN","2overN","percentile"],axis=1).fillna(0)
test_business = pd.read_csv("LDA Modelling/results/test/business_FeatureVec_test.csv").drop(["business_id","review_id","user_id","1overN","2overN","percentile"],axis=1).fillna(0)
stacked= test_business[['attributes.Ambience.casual','attributes.BikeParking','attributes.BusinessAcceptsCreditCards','attributes.BusinessParking.street','attributes.HasTV','attributes.RestaurantsDelivery','attributes.GoodForMeal.breakfast','attributes.GoodForMeal.brunch','attributes.GoodForMeal.dessert','attributes.GoodForMeal.dinner','attributes.GoodForMeal.latenight','attributes.GoodForMeal.lunch','attributes.Alcohol','attributes.NoiseLevel']].stack()
test_business[['attributes.Ambience.casual','attributes.BikeParking','attributes.BusinessAcceptsCreditCards','attributes.BusinessParking.street','attributes.HasTV','attributes.RestaurantsDelivery','attributes.GoodForMeal.breakfast','attributes.GoodForMeal.brunch','attributes.GoodForMeal.dessert','attributes.GoodForMeal.dinner','attributes.GoodForMeal.latenight','attributes.GoodForMeal.lunch','attributes.Alcohol','attributes.NoiseLevel']]=pd.Series(stacked.factorize()[0], index=stacked.index).unstack()
#test_business = test_business.loc[:,["topic_0","topic_1","topic_2","topic_3","topic_4","topic_5","topic_6","topic_7","topic_8","topic_9","topic_10","topic_11","topic_12","topic_13","topic_14","topic_15","topic_16","topic_17","topic_18","topic_19","business_stars","is_open","attributes.Ambience.casual","attributes.BikeParking","attributes.BusinessAcceptsCreditCards","attributes.BusinessParking.street","attributes.HasTV","attributes.RestaurantsDelivery","attributes.GoodForMeal.breakfast","attributes.GoodForMeal.brunch","attributes.GoodForMeal.dessert","attributes.GoodForMeal.dinner","attributes.GoodForMeal.latenight","attributes.GoodForMeal.lunch","attributes.Alcohol","attributes.NoiseLevel","attributes.RestaurantsPriceRange2"]]

#reviews = pd.read_csv("user_business_reviewVec.csv")['review_id']

test_rating_csv = pd.read_csv("Dataset/PA/Restaurants/test/PA_test_yelp_academic_dataset_review.csv")
test_rating_csv_stars = test_rating_csv['stars']
encoded_rating = to_categorical(np.array(list(test_rating_csv_stars)))
encoded_rating_test = encoded_rating[:,1:]

In [0]:
valid_inputs = [valid_user,valid_business]
predictions_valid = stackedModel.model.predict(valid_inputs)
from sklearn.metrics import mean_squared_error

score_valid = np.sqrt(mean_squared_error(encoded_rating_valid,predictions))
print("RMSE valid: ",score_valid)

RMSE valid:  0.39890584


In [0]:
train_inputs = [train_user,train_business]
predictions = stackedModel.model.predict(train_inputs)
from sklearn.metrics import mean_squared_error

score_valid = np.sqrt(mean_squared_error(encoded_rating_,predictions))

print("RMSE train: ",score_valid)

RMSE train:  0.40145263


In [0]:
test_inputs = [test_user,test_business]
predictions_test = stackedModel.model.predict(test_inputs)
from sklearn.metrics import mean_squared_error

score_test = np.sqrt(mean_squared_error(encoded_rating_test,predictions))
print("RMSE test: ",score_test)

RMSE test:  0.39853543


In [0]:
score = stackedModel.model.evaluate(test_inputs,encoded_rating_test)
print(score)

65591/65591 [==============================] - 2s 33us/step
[1.6916372785155165, 0.33789696757404236]


In [0]:
test_inputs = [test_user,test_business]
predictions = stackedModel.model.predict(test_inputs)

In [0]:
score = stackedModel.model.evaluate(test_inputs,encoded_rating_test)

65591/65591 [==============================] - 2s 28us/step


In [0]:
print(score)

[1.3845180452857757, 0.4405939839257242]